In [38]:
import os
import numpy as np
import pandas as pd
import base64

import requests
from bs4 import BeautifulSoup
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [39]:
data_path = r'C:\Users\Narayanan\Desktop\python\scrapper\scrap_details.xlsx'
save_folder_path = r'D:\data'
driver_path = r'C:/Users/Narayanan/Desktop/python/scrapper/chromedriver.exe'

In [40]:
data = pd.read_excel(data_path)

In [41]:
data.tail()

,class,keyword
23,Sleeping,Sleeping in Train
24,Sleeping,Sleeping in bus
25,Sleeping,Sleeping in car
26,Sleeping,Sleeping in Pedestrian Walk
27,Sleeping,Sleeping in flight


In [42]:
data = data.groupby('class')['keyword'].apply(list).to_dict()

In [43]:
def get_image_links_from_search(query, num_pages=1):
    image_links = []
    
    service = Service(executable_path= driver_path)
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    browser = webdriver.Chrome(service=service, options=options)

    for page in range(num_pages):
        start = page * 50
        url = f"https://www.google.com/search?q={query}&tbm=isch&start={start}"

        browser.get(url)
        page_source = browser.page_source

        soup = BeautifulSoup(page_source, 'html.parser')

        for img in soup.find_all('img'):
            src = img.get('src')
            if src:
                image_links.append(src)

    browser.quit()
    image_links = list(set(image_links))
    
    return image_links


In [44]:
all_links = dict()
for query_class, query in data.items():
    class_link = []
    for i in query:
        fetched_links = get_image_links_from_search(i, 3)
        class_link.extend(fetched_links)
    all_links[query_class] = list(set(class_link))

In [45]:
def download_image(url, file_path):
    if url.startswith('data'):
        head, data = url.split(',', 1)
        file_ext = head.split(';')[0].split('/')[1]
        
        plain_data = base64.b64decode(data)

        with open(file_path, 'wb') as file:
            file.write(plain_data)
    try:
        response = requests.get(url)
        response.raise_for_status()

        with open(file_path, 'wb') as file:
            file.write(response.content)
    except:
        pass

In [46]:
for class_name, img_url in all_links.items():
    count = len(img_url)
    
    path = os.path.join(save_folder_path, class_name)
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
    
    for i in range(len(img_url)):
        file_path = path + '\\' + class_name + '_' + f"{i:05d}.jpg"
        download_image(img_url[i], file_path)